<a href="https://colab.research.google.com/github/Soguy-fr/Test-effacer/blob/main/Velib_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initial setup

In [ ]:
import pandas as pd
import requests
from geopy.distance import geodesic


# URL JSON files
url_station_status = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json"
url_station_information  ="https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json"

#Get station_status data into Panda DataFrame
To find out how many bikes and terminals are available at each station :
* **station_id**: unique station identification number. This number identifies the station within the Vélib' Métropole service.
* **is_installed**: binary variable indicating whether the station is. The station has already been deployed (1) or is still being deployed (0). 
* **is_renting**: binary variable indicating whether the station can rent bicycles (is_renting=1 if the station status is Operative).
* **is_returning**: binary variable indicating whether the station can receive bikes (is_renting=1 if the station status is Operative).
* **last_reported**: date of last update 
* **numBikesAvailable**: number of bikes available
* **numDocksAvailable**: number of docking stations available
* **num_bikes_available**: number of bikes available
* **num_bikes_available_types**: number of bikes available, distinguishing between mechanical and electric Vélib'.
---






In [ ]:
# Get station_status
response = requests.get(url_station_status)
data = response.json()

# Create Dataframe from Json
station_status = pd.DataFrame(data["data"]["stations"])

station_status

,stationCode,station_id,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported
0,16107,213688169,3,3,"[{'mechanical': 1}, {'ebike': 2}]",32,32,1,1,1,1686047322
1,31104,653222953,6,6,"[{'mechanical': 4}, {'ebike': 2}]",19,19,1,1,1,1686047424
2,13007,66491398,1,1,"[{'mechanical': 1}, {'ebike': 0}]",47,47,1,0,0,1686047485
3,9020,36255,1,1,"[{'mechanical': 0}, {'ebike': 1}]",19,19,1,1,1,1686047167
4,12109,37815204,4,4,"[{'mechanical': 4}, {'ebike': 0}]",25,25,1,1,1,1686047226
...,...,...,...,...,...,...,...,...,...,...,...
1459,18023,54000604,2,2,"[{'mechanical': 1}, {'ebike': 1}]",32,32,1,1,1,1686047391
1460,18026,43247738,0,0,"[{'mechanical': 0}, {'ebike': 0}]",31,31,1,1,1,1686047394
1461,18024,102311820,1,1,"[{'mechanical': 0}, {'ebike': 1}]",34,34,1,1,1,1686047275
1462,42004,368766689,33,33,"[{'mechanical': 29}, {'ebike': 4}]",3,3,1,1,1,1686047550


# Get station information

* **capacity**: number of terminals in the station
* **lat**: station latitude (in WGS84 format)
* **lon**: station longitude (in WGS84 format)
* **name**: station name
* **station_id**: unique identifier associated with the station. This number identifies the station within the Vélib' Métropole service.

In [ ]:
# Get url_station_information
response = requests.get(url_station_information)
data = response.json()

# Create Dataframe from Json
station_information = pd.DataFrame(data["data"]["stations"])


#Some tests on the data

In [ ]:
#print(df.describe())
#print(station_status.head())
#print(station_information.head())
#column_names = station_status.columns
#print(column_names)


# Your GPS coordinates
your_latitude = 48.8616064374412
your_longitude = 2.3337732420557638

# Coordinates of the first station in the table
station_latitude = station_information['lat'][0]
station_longitude = station_information['lon'][0]

# Calcul de la distance entre les deux points
distance = geodesic((your_latitude, your_longitude), (station_latitude, station_longitude)).kilometers

print("Distance between your position and the first station of the list : {:.2f} kilometers".format(distance))


Distance between your position and the first station : 4.29 kilomètres


#Calculate the GSP coordinate of the nearest bike station from your position and from the destination

In [ ]:
# Function for calculating the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

# Your GPS coordinates
current_latitude = 48.8616064374412
current_longitude = 2.3337732420557638

# Destination GPS coordinates 
destination_latitude = 48.845003
destination_longitude = 2.328524


# Create a column with the distance between the station and your GPS coordinates
station_information['distance_from_current'] = station_information.apply(lambda row: calculate_distance(current_latitude, current_longitude, row['lat'], row['lon']), axis=1)

# Create a column with the distance between the station and the GPS coordinates of your destination
station_information['distance_from_destination'] = station_information.apply(lambda row: calculate_distance(destination_latitude, destination_longitude, row['lat'], row['lon']), axis=1)

# Nearest station from your position
closest_station_from_current = station_information.loc[station_information['distance_from_current'].idxmin()]

# Nearest station from your destination
closest_station_from_destination = station_information.loc[station_information['distance_from_destination'].idxmin()]

# Display information about the nearest station from your current position
print("The nearest station of your current position is:")
print("Name:", closest_station_from_current['name'])
print("Distance:", closest_station_from_current['distance_from_current'], "kilometres")

# Display information about the nearest station from your destination
print("The nearest station from your destination is:")
print("Name:", closest_station_from_destination['name'])
print("Distance:", closest_station_from_destination['distance_from_destination'], "kilometres")

#Is there available bike at the station

In [ ]:
# Find the station ID of the closest station from current position
closest_station_id = closest_station_from_current['station_id']

# Get the row in station_status corresponding to the closest station
closest_station_status = station_status[station_status['station_id'] == closest_station_id]

# Get the number of available bikes for the closest station
num_bikes_available = closest_station_status['num_bikes_available'].values[0]

print("Number of available bikes at the closest station:", num_bikes_available)


Number of available bikes at the closest station: 19


#Function to find the nearest stations
Now that we are able to find a station and the number of available bike, let's create a function which give us the nearest station of GPS point, with at least 1 available bike

In [ ]:
def find_nearest_station(latitude, longitude):
    # Create a column with the distance between the station and the input GPS coordinates
    station_information['distance_from_input'] = station_information.apply(
        lambda row: calculate_distance(latitude, longitude, row['lat'], row['lon']),
        axis=1
    )
    
    # Find the nearest station from the input GPS coordinates
    nearest_station = station_information.loc[station_information['distance_from_input'].idxmin()]
    
    # Get the station ID of the nearest station
    nearest_station_id = nearest_station['station_id']
    
    # Get the row in station_status corresponding to the nearest station
    nearest_station_status = station_status[station_status['station_id'] == nearest_station_id]
    
    # Get the number of available bikes for the nearest station
    num_bikes_available = nearest_station_status['num_bikes_available'].values[0]
    
    # Return the nearest station, its name, and the number of available bikes
    return nearest_station, nearest_station['name'], num_bikes_available

# Example usage:
input_latitude = 48.8616064374412
input_longitude = 2.3337732420557638

nearest_station, station_name, num_bikes_available = find_nearest_station(input_latitude, input_longitude)

print("Nearest Station:", station_name)
print("Number of available bikes:", num_bikes_available)


Nearest Station: Saint-Honoré - Musée du Louvre
Number of available bikes: 19
